In [1]:
from openai import OpenAI

In [2]:
import wikipediaapi


def query_research_url(input):
    query = input['query']
    wiki = wikipediaapi.Wikipedia(
        user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"
    )
    page = wiki.page(query)
    if page.exists():
        return page.fullurl
    else:
        return None


functions = [
    {
        "type": "function",
        "function": {
            "name": "QueryReasearchUrl",
            "description": "to find widipedia url for query",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The name of company"
                    }
                },
                "required": ["query"]
            }
        }
    }
]

In [3]:
client = OpenAI()
assistant = client.beta.assistants.create(
    name="Research Assistant",
    model="gpt-4o-mini",
    instructions="Research about XZ backdoor",
    tools=functions
)
assistant

Assistant(id='asst_ZRMreUiZNNIZNpjBleZjNA4I', created_at=1726215656, description=None, instructions='Research about XZ backdoor', metadata={}, model='gpt-4o-mini', name='Research Assistant', object='assistant', tools=[FunctionTool(function=FunctionDefinition(name='QueryReasearchUrl', description='to find widipedia url for query', parameters={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'The name of company'}}, 'required': ['query']}, strict=False), type='function')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)

In [4]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want to know about XZ backdoor"
        }
    ]
)
thread

Thread(id='thread_itGtsd7s7NlRoYgYgIVpXf2W', created_at=1726215658, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [6]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)
run

Run(id='run_p6jp5B6kZN32kN5FcH17hcZC', assistant_id='asst_ZRMreUiZNNIZNpjBleZjNA4I', cancelled_at=None, completed_at=None, created_at=1726215670, expires_at=1726216270, failed_at=None, incomplete_details=None, instructions='Research about XZ backdoor', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-mini', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_itGtsd7s7NlRoYgYgIVpXf2W', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='QueryReasearchUrl', description='to find widipedia url for query', parameters={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'The name of company'}}, 'required': ['query']}, strict=False), type='function')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})

In [5]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )
def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")

In [10]:
get_run(run.id, thread.id).status

'requires_action'

In [9]:
get_messages(thread.id)

user: I want to know about XZ backdoor


In [11]:
query_research_url("XZ backdoor")

'https://en.wikipedia.org/wiki/XZ_Utils_backdoor'